<a href="https://colab.research.google.com/github/salim-hbk/ai-ml/blob/main/VGG_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

Lets explore and Build VGG model. Using custom Model architecture.
<img src='https://raw.githubusercontent.com/anhtuan85/TensorFlow-Advanced-Techniques-Specialization/98365012056370410d0280aeb0bcb454d1f65f82/Course%201%20-%20Custom%20Models%2C%20Layers%2C%20and%20Loss%20Functions%20with%20TensorFlow/Week%204/VGG.png' />

**Creating named Variables**

you will see the use of the Python function vars(). This will allow you to use a for loop to define and set multiple variables with a similar name, such as var1, var2, var3.

Please go through the following examples to get familiar with vars(), as you will use it when building the VGG model.

You'll start by defining a class MyClass

It contains one variable var1.

Create an object of type MyClass.

Python classes have an attribute called __dict__.

__dict__ is a Python dictionary that contains the object's instance variables and values as key value pairs.

In [2]:
class MyClass:
  def __init__(self):
    self.var1=1

myClass = MyClass()

myClass.__dict__

{'var1': 1}


If you call vars() and pass in an object, it will call the object's __dict__ attribute, which is a Python dictionary containing the object's instance variables and their values as ke

In [3]:
vars(myClass)

{'var1': 1}

In [4]:
myClass.var2 = 2
myClass.var3 = 3
vars(myClass)

{'var1': 1, 'var2': 2, 'var3': 3}

In [5]:
vars(myClass)['var3'] = 33
myClass.__dict__['var3'] = 44
myClass.__dict__

{'var1': 1, 'var2': 2, 'var3': 44}

Why this is helpful!

You may be wondering why you would need another way to access an object's instance variables.

Notice that when using vars(), you can now pass in the name of the variable var3 as a string.
What if you plan to use several variables that are similarly named (var4, var5 ... var9) and wanted a convenient way to access them by incrementing a number?

In [6]:
for i in range(1, 10):
  vars(myClass)[f'var{i}'] = i

vars(myClass)

{'var1': 1,
 'var2': 2,
 'var3': 3,
 'var4': 4,
 'var5': 5,
 'var6': 6,
 'var7': 7,
 'var8': 8,
 'var9': 9}

Create a generic VGG block (TODO)

The VGG Network has blocks of layers, where each block has a varied number of layers.

In order to create blocks of layers that have a customizable number of conv2D layers, you'll define a class Block, which can generate a customizable block of layers

__init__

In the constructor __init__, store the conv2D parameters and also define the number of conv2D layers using the parameters passed into __init__.

Store the filters, kernel_size, and repetitions as class variables so that they can be used later in the call function.
Using a for loop, define a number of Conv2D Conv2D layers, based on the number of repetitions desired for this block.

You can define each conv2D layer using vars and string formatting to create conv2D_0, conv2D_1, conv2D_3 etc.
Set these four parameters of Conv2D:

filters

kernel_size

activation: set this to 'relu'
padding: set this to 'same' (default pading is 'valid').
Define the MaxPool2D layer that follows these Conv2D layers.

Set the following parameters for MaxPool2D:
pool_size: this will be a tuple with two values.
strides: this will also be a tuple with two values.
call
In call, you will connect the layers together.

The 0-th conv2D layer, conv2D_0, immediately follows the inputs.
For conv2D layers 1,2 and onward, you can use a for loop to connect conv2D_1 to conv2D_0, and connect conv2D_2 to conv2D_1, and so on.
After connecting all of the conv2D_i layers, add connect the max_pool layer and return the max_pool layer.

In [7]:
class Block(tf.keras.Model):
  def __init__(self, filters, kernel, repetitions, pool_size=2, strides=2):
    super(Block, self).__init__()
    self.filters = filters
    self.kernel_size = kernel
    self.repetitions = repetitions

    for i in range(self.repetitions):
      vars(self)[f'conv2d_{i}'] = tf.keras.layers.Conv2D(self.filters, self.kernel_size, activation='relu', padding='same')
    
    self.max_pool = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=strides)

  def call(self, inputs):
    conv2d0 = vars(self)['conv2d_0']
    x= conv2d0(inputs)

    for i in range(1, self.repetitions):
      x = vars(self)[f'conv2d_{i}'](x)

    max_pool = self.max_pool(x)
    return max_pool


Create the Custom VGG network (TODO)
This model stack has a series of VGG blocks, which can be created using the Block class that you defined earlier.

__init__

Recall that the __init__ constructor of Block takes several function parameters,
filters, kernel_size, repetitions: you'll set these.
kernel_size and strides: you can use the default values.
For blocks a through e, build the blocks according to the following 

specifications:

block_a: 64 filters, kernel_size 3, repetitions 2

block_b: 128 filters, kernel_size 3, repetitions 2

block_c: 256 filters, kernel_size 3, repetitions 3

block_d: 512 filters, kernel_size 3, repetitions 3

block_e: 512 filters, kernel_size 3, repetitions 3

After block 'e', add the following layers:

flatten: use Flatten.

fc: create a fully connected layer using Dense. Give this 256 units, and a 'relu' activation.
classifier: create the classifier using a Dense layer. The number of units equals the number of classes. For multi-class classification, use a 'softmax' activation.
call
Connect these layers together using the functional API syntax:

inputs
block_a
block_b
block_c
block_d
block_e
flatten
fc
classifier
Return the classifier layer.

In [8]:
class MyVgg(tf.keras.Model):
  def __init__(self, num_classes):
    super(MyVgg, self).__init__()
    self.block_a = Block(filters=64,kernel=3,repetitions=2)
    self.block_b = Block(filters=128,kernel=3,repetitions=2)
    self.block_c = Block(filters=256,kernel=3,repetitions=3)
    self.block_d = Block(filters=512,kernel=3,repetitions=3)
    self.block_e = Block(filters=512,kernel=3,repetitions=3)

    self.flatten = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(256, 'relu')
    self.classifier = tf.keras.layers.Dense(num_classes, 'softmax')

  def call(self, inputs):
    # Chain all the layers one after the other
    x = self.block_a(inputs)
    x = self.block_b(x)
    x = self.block_c(x)
    x = self.block_d(x)
    x = self.block_e(x)
    x = self.flatten(x)
    x = self.fc(x)
    x = self.classifier(x)
    return x



Load data and train the VGG network (Optional)

You can now load the dataset and proceed to train your VGG network.

In [9]:
import tensorflow_datasets as tfds

In [10]:
datasets= tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')
vgg = MyVgg(num_classes=2)

vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def preprocess(features):
  images = tf.image.resize(features['image'], (224, 224))
  return tf.cast(images, tf.float32) / 255., features['label']

dataset = datasets.map(preprocess).batch(32)

vgg.fit(dataset, epochs=10)

Shuffling and writing examples to data/cats_vs_dogs/4.0.0.incomplete4LYYDA/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to data/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Epoch 1/10
727/727 [==============================] - 95s 118ms/step - loss: 0.6919 - accuracy: 0.5298
Epoch 2/10
727/727 [==============================] - 87s 120ms/step - loss: 0.6688 - accuracy: 0.6062
Epoch 3/10
727/727 [==============================] - 88s 121ms/step - loss: 0.6378 - accuracy: 0.6473
Epoch 4/10
727/727 [==============================] - 88s 121ms/step - loss: 0.6149 - accuracy: 0.6652
Epoch 5/10
727/727 [==============================] - 88s 121ms/step - loss: 0.6011 - accuracy: 0.6758
Epoch 6/10
727/727 [==============================] - 87s 120ms/step - loss: 0.5918 - accuracy: 0.6833
Epoch 7/10
727/727 [==============================] - 88s 120ms/step - loss: 0.5846 - accuracy: 0.6894
Epoch 8/10
727/727 [==============================] - 88s 121ms/step - loss: 0.5785 - accuracy: 0.6958
Epoch 9/10
727/727 [==============================] - 88s 121ms/